In [1]:
import sys

In [2]:
sys.path.append('../')

In [3]:
from model.model import FineTuner
from model.dataloader import DataModule

/home2/shivprasad.sagare/miniconda3/envs/xalign_role/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dm_args = {
    'train_path': '../data/xalign_HRT_only/train.csv',
    'val_path': '../data/xalign_HRT_only/val.csv',
    'test_path': '../data/xalign_HRT_only/test.csv',
    'tokenizer_name_or_path': 'google/mt5-small',
    'max_source_length': 384,
    'max_target_length': 128,
    'train_batch_size': 1,
    'val_batch_size': 1,
    'test_batch_size': 1,
}

In [5]:
dm = DataModule(**dm_args)

we added S|, P|, O| to the tokenizer


In [6]:
model_args = {
    'learning_rate': 3e-5,
    'model_name_or_path': 'google/mt5-small',
    'eval_beams': 4,
    'tgt_max_seq_len': 128,
    'tokenizer': dm.tokenizer,
}

In [7]:
model = FineTuner(**model_args)

Some weights of MT5ForConditionalGeneration were not initialized from the model checkpoint at google/mt5-small and are newly initialized: ['encoder.role_embed.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
dm.setup(stage='test')

In [13]:
for batch in dm.test_dataloader():
    batch1 = batch
    break

In [16]:
input_ids, attention_mask, role_ids, labels = batch1['input_ids'], batch1['attention_mask'],  batch['role_ids'], batch1['labels']

In [17]:
outputs = model(input_ids, attention_mask, role_ids=role_ids, labels=labels)

In [ ]:
labels.shape

torch.Size([1, 128])

In [18]:
outputs.logits.shape

torch.Size([1, 128, 250103])

In [1]:
!export WANDB_API_KEY=bf6eddaca0cddb4d9e70aa37fb5ef56202d7ef74

import wandb

run = wandb.init()

artifact = run.use_artifact('shivprasad/xalign/model-2w17e6wd:v0', type='model')

artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shivprasad (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-08-24 22:22:56.846029: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-08-24 22:22:56.846841: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


wandb: Downloading large artifact model-2w17e6wd:v0, 3444.44MB. 1 files... Done. 0:0:0


In [8]:
import torch

In [9]:
checkpoint = torch.load(r'artifacts/model-2w17e6wd:v0/model.ckpt')
print(checkpoint.keys())

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])


In [10]:
sd = checkpoint['state_dict']

In [11]:
model.load_state_dict(sd)

<All keys matched successfully>